In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
#load encoder,scaler pickle,onehot
from tensorflow.keras.models import load_model
model=load_model('model.h5')
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)
with open('label_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
input_data={
    "CreditScore": 750,
    "Geography":'France',
    "Gender": 'Male',
    "Age": 35,
    "Tenure": 5,
    "Balance": 12000.50,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 60000.00,
    
  }

In [10]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,France,Male,35,5,12000.5,2,1,1,60000.0


In [11]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,France,1,35,5,12000.5,2,1,1,60000.0


In [12]:
geo_encoded_array=label_encoder_geo.transform(input_df[['Geography']]).toarray()
geo_encoded_array

array([[1., 0., 0.]])

In [13]:
geo_encoded_df=pd.DataFrame(geo_encoded_array,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,1,35,5,12000.5,2,1,1,60000.0,1.0,0.0,0.0


In [15]:
input_scaler=scaler.transform(input_df)
input_scaler

array([[ 1.0206745 ,  0.91324755, -0.37056859, -0.00134472, -1.02633068,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
prediction=model.predict(input_scaler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array([[0.00557228]], dtype=float32)

In [17]:
prediction_proba=prediction[0][0]
prediction_proba

0.005572284

In [18]:
if prediction_proba >0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
